<a href="https://colab.research.google.com/github/ge96lip/Quantum-Computing/blob/main/QC_Shor's_Factorization_Algorithm_Laura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites

In [1]:
!pip install qiskit
!pip install pylatexenc
!pip install qiskit_aer
!pip install qiskit-ibm-runtime
!pip install qiskit qiskit-ibm-runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=f25cd0f748cc43243f42071f75e6d90cbffd9e1f439573a0abf5bf9d52aefb33
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit import Qubit
from qiskit.circuit.library import XGate, CSwapGate, QFT
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, SamplerOptions
from qiskit.visualization import plot_histogram

import matplotlib.pyplot as plt
import math
import time

# Loading your IBM Q account
# Token KTH: 86ad51ed8e455dea40b8ac15c7b6359cf38cb674fdb3072f3d2e6211debf389fcfe547c54aad314614600bbe71f3d83572bc7059184a3bb11feae345ec347801
# Token TUM: b0f87c37cccc19fb5948a66803edbb2e8f4f598db8f595934199d7ff9aa28946f7cb734ced928a33344a1477fe7b5fb5e75572f4694d0be2e81c7b3dbadff8f7
QiskitRuntimeService.save_account(channel="ibm_quantum", token='b0f87c37cccc19fb5948a66803edbb2e8f4f598db8f595934199d7ff9aa28946f7cb734ced928a33344a1477fe7b5fb5e75572f4694d0be2e81c7b3dbadff8f7', overwrite=True)

# Shor's Factorization Algorithm

In [3]:
def ShorLogic(N, repeat_period_candidates_list, coprime):
    factor_candidates_list = []
    for repeat_period_candidates in repeat_period_candidates_list:
        #print('Repeat period candidates: ', repeat_period_candidates)
        factor_candidates = []
        for repeat_period in repeat_period_candidates:
            # Given the repeat period, find the actual factors
            ar2 = pow(coprime, repeat_period / 2.0)
            factor1 = int(gcd(N, ar2 - 1))
            factor2 = int(gcd(N, ar2 + 1))
            factor_candidates.append([factor1, factor2])
        factor_candidates_list.append(factor_candidates)
    return factor_candidates_list

In [4]:
def gcd(a, b):
    # return the greatest common divisor of a,b
    while b:
        m = a % b
        a = b
        b = m
    return a

In [5]:
def check_single_result(N, factor_candidates):
    for factors in factor_candidates:
        if factors[0] * factors[1] == N:
            if factors[0] != 1 and factors[1] != 1:
                # Success!
                return factors
    # Failure
    return None

def check_result(N, factor_candidates_list):
    factors_list = []
    for factors_candidates in factor_candidates_list:
        factors = check_single_result(N, factors_candidates)
        factors_list.append(factors)
    return factors_list

In [6]:
def Shor(N, coprime, classical=False, simulate=True, vis=False, opt_for_vis=False, shots=1):
    # Quantum part
    if classical:
        # Classical alternative for comparision
        repeat_period_candidates_list, elapsed_time_quantum, start_time_classical = ShorNoQPU(N, coprime, shots)
    else:
        # Quantum part
        repeat_period_candidates_list, elapsed_time_quantum, start_time_classical = ShorQPU(N, coprime, simulate, vis, opt_for_vis, shots)

    # Classical part
    factor_candidates_list = ShorLogic(N, repeat_period_candidates_list, coprime)

    return [check_result(N, factor_candidates_list), elapsed_time_quantum, start_time_classical]

In [7]:
def ShorNoQPU(N, coprime, shots=1):
    # Classical replacement for the quantum part of Shor
    start_time = time.time()
    repeat_period_candidates_list = []
    for i in range(shots):
        repeat_period_candidates = []
        work = 1
        precision_bits = math.ceil(math.log2(N))
        max_loops = pow(2, precision_bits)
        for iter in range(max_loops):
            work = (work * coprime) % N
            if work == 1: # found a repeat period
                repeat_period_candidates.append(iter + 1)
        repeat_period_candidates_list.append(repeat_period_candidates)
    end_time = time.time()
    elapsed_time = end_time - start_time
    start_time = time.time()
    return [repeat_period_candidates_list, elapsed_time, start_time]

In [8]:
def ShorQPU(N, coprime, simulate=True, vis=False, opt_for_vis=False, shots=1):
    # Quantum part of Shor's algorithm
    # For this implementation, the coprime must be 2.
    coprime = 2;
    return ShorQPU_WithModulo(N, coprime, simulate, vis, opt_for_vis, shots)

    '''
    # For some numbers (like 15 and 21) the "mod" in a^xmod(N)
    # is not needed, because a^x wraps neatly around. This makes the
    # code simpler, and much easier to follow.
    if N == 15 or N == 21:
        return ShorQPU_WithoutModulo(N, coprime, simulate, opt_for_vis, shots)
    else:
        return ShorQPU_WithModulo(N, coprime, simulate, vis, opt_for_vis, shots)
    '''

In [9]:
def RollLeft(qc, work, num_shifts, control):
    # Ensure we're not out of range for work qubits
    for j in range(work.size - num_shifts-1,-1,-1):
        # Apply CSWAP between work[j] <-> work[j + num_shifts]
        qc.append(CSwapGate(), [control, work[j], work[j + num_shifts]])
    return qc

In [10]:
# In case our QPU read returns a "signed" negative value,
# convert it to unsigned.
def read_unsigned(N, coprime, qc, qreg, creg, simulate=True, shots=1):
    qc.measure(qreg, creg)

    if simulate:
        # Simulate the circuit using the Aer simulator
        simulator = AerSimulator()

        start_time = time.time()
        job = simulator.run(transpile(qc, simulator), shots=shots)
        end_time = time.time()
        result = job.result()
        counts = result.get_counts(qc)

    else:
        # Run the circuit on a IBM quantum computer
        service = QiskitRuntimeService()
        #backend = service.backend("ibm_sherbrooke")
        backend = service.backend("ibm_brisbane")  #backend = service.least_busy(operational=True, simulator=False, min_num_qubits=qc.num_qubits)
        print("Backend: ",backend.name)
        options = SamplerOptions(environment={"job_tags": [f"N={N}/a={coprime}/s={shots}"]})
        sampler = Sampler(mode=backend, options=options)

        # Uncomment to use already stored job ##################################
        #job_id = 'cwr9800997wg008xec20'
        #job = service.job(job_id)
        ########################################################################

        start_time = time.time()
        # Comment out to use already stored job ################################
        job = sampler.run([transpile(qc, backend)], shots=shots)
        job_id = job.job_id()
        ########################################################################
        end_time = time.time()
        print("Job ID: ", job_id)

        result = job.result()[0].data[creg.name]
        counts = result.get_counts()

    # Expand the counts into an array
    measurement_results = []
    for outcome, count in counts.items():
        measurement_results.extend([outcome] * count)

    elapsed_time_quantum = end_time - start_time

    # Extract value of measurement outcomes
    read_result_list = []
    for result in measurement_results:
        read_result_list.append(int(result,2) & ((1 << qreg.size) - 1))

    return [read_result_list, elapsed_time_quantum]

In [11]:
def estimate_num_spikes(spike_list, spike_range):
    candidates_list = []
    for spike in spike_list:
        if spike < spike_range / 2:
            spike = spike_range - spike
        best_error = 1.0
        e0 = 0
        e1 = 0
        e2 = 0
        actual = spike / spike_range
        candidates = []
        for denominator in range(1,spike):
            numerator = round(denominator * actual)
            estimated = numerator / denominator
            error = abs(estimated - actual)
            e0 = e1
            e1 = e2
            e2 = error
            # Look for a local minimum which beats our current best error
            if e1 <= best_error and e1 < e0 and e1 < e2:
                repeat_period = denominator - 1
                candidates.append(repeat_period)
                best_error = e1
        candidates_list.append(candidates)
    return candidates_list

In [12]:
# This is the short/simple version of ShorQPU() where we can perform a^x and
# don't need to be concerned with performing a quantum int modulus.

def ShorQPU_WithoutModulo(N, coprime, simulate=True, vis=False, opt_for_vis=False, shots=1):

    # Define number of required qubits
    N_bits = math.ceil(math.log2(N))
    work_qubits = N_bits + 1
    precision_qubits = N_bits

    # Set up the QPU and the working registers
    work = QuantumRegister(work_qubits, name="work")
    precision = QuantumRegister(precision_qubits, name="precision")
    classical = ClassicalRegister(precision_qubits, name="precision_measure")
    qc = QuantumCircuit(work, precision, classical)

    # Initialization: set the working register to state |1>
    # and the precision register to state |0>
    # put the precision register into superposition using Hadamard gate
    qc.initialize(1, work)
    qc.initialize(0, precision)
    qc.h(precision)

    if opt_for_vis: qc.barrier()

    # Perform 2^x for all possible values of x in superposition
    for iter in range(precision_qubits):
        num_shifts = 1 << iter
        if num_shifts < work_qubits:
            qc = RollLeft(qc, work, num_shifts, precision[iter])
            if opt_for_vis: qc.barrier()

    # Quantum Fourier Transform on precision register
    qc.append(QFT(precision_qubits), precision)

    if opt_for_vis: qc.barrier()

    if vis: print(qc)

    print('Circuit depth: ', qc.depth())
    print('Number of qubits: ', qc.num_qubits)

    [read_result_list, elapsed_time_quantum] = read_unsigned(N, coprime, qc, precision, classical, simulate, shots)
    #print('QPU read result: ', read_result_list)

    start_time_classical = time.time()

    repeat_period_candidates_list = estimate_num_spikes(read_result_list, 1 << precision_qubits)
    #print('Repeat period candidates list: ', repeat_period_candidates_list)

    return [repeat_period_candidates_list, elapsed_time_quantum, start_time_classical]

In [13]:
def add_int(qc, qreg, N, conditions):
    reverse_to_subtract = False
    if N == 0:
        return
    elif N < 0:
        N = -N
        reverse_to_subtract = True
    ops = []
    add_val = int(N)
    condition_mask = (1 << len(qreg)) - 1

    add_val_mask = 1
    while add_val_mask <= add_val:
        cmask = condition_mask & ~(add_val_mask - 1)
        if add_val_mask & add_val:
            add_shift_mask = 1 << (len(qreg) - 1)
            while add_shift_mask >= add_val_mask:
                cmask &= ~add_shift_mask
                ops.append((add_shift_mask, cmask))
                add_shift_mask >>= 1
        condition_mask &= ~add_val_mask
        add_val_mask <<= 1
    if reverse_to_subtract:
        ops.reverse()
    for inst in ops:
        op_qubits = []
        for i in range(len(qreg)):
            if inst[1] & (1 << i):
                op_qubits.append(qreg[i])
        for i in range(len(qreg)):
            if inst[0] & (1 << i):
                op_qubits.append(qreg[i])
        qc = multi_cx(qc, op_qubits, conditions)

    return qc


def multi_cx(qc, qubits, conditions=None, do_cz=False):
    ## This will perform a CCCCCX with as many conditions as we want
    ## The last qubit in the list is the target.

    target = qubits[-1]
    conds = qubits[:-1]
    ops = []

    controll_qubits = []

    for i in range(len(conds)):
        controll_qubits.append(conds[i])
    if conditions != None:
        for i in range(len(conditions)):
            controll_qubits.append(conditions[i])

    if len(controll_qubits) == 0:
        qc.x(target)
    else:
        qc.mcx(controll_qubits, target)

    return qc

In [14]:
# This is the complicated version of ShorQPU() where we DO
# need to be concerned with performing a quantum int modulus.
# That's a complicated operation, and it also requires us to
# do the shifts one at a time.
def ShorQPU_WithModulo(N, coprime, simulate=True, vis=False, opt_for_vis=False, shots=1):
    max_value = 1
    mod_engaged = False

    # Define number of required qubits
    N_bits = math.ceil(math.log2(N))
    work_qubits = N_bits + 1
    precision_qubits = N_bits
    scratch_qubits = 1

    # Set up the QPU and the working registers
    work = QuantumRegister(work_qubits, name="work")
    precision = QuantumRegister(precision_qubits, name="precision")
    scratch = QuantumRegister(scratch_qubits, name="scratch")
    classical = ClassicalRegister(precision_qubits, name="precision_measure")
    qc = QuantumCircuit(work, precision, scratch, classical)

    # Initialization: set the working register to state |1>
    # and the precision register to state |0>
    # put the precision register into superposition using Hadamard gate
    qc.initialize(1, work)
    qc.initialize(0, precision)
    qc.initialize(0, scratch)
    qc.h(precision)

    if opt_for_vis: qc.barrier()

    N_sign_bit = work[work_qubits - 1]

    for iter in range(precision_qubits):
        condition_qubit = precision[iter]  # Get the condition qubit

        shifts = 1 << iter
        for shift in range(shifts):
            qc = RollLeft(qc, work, 1, precision[iter]) # multiply by the coprime 2
            if opt_for_vis: qc.barrier()
            max_value <<= 1

            if max_value >= N:
                mod_engaged = True

            if mod_engaged:
                qc = add_int(qc, work, -N, [condition_qubit]) # Substract N from working register, causing this to go negative if work < N
                if opt_for_vis: qc.barrier()
                qc.ccx(N_sign_bit, condition_qubit, scratch) # Skim off the sign bit
                if opt_for_vis: qc.barrier()
                qc = add_int(qc, work, N, [scratch[0], condition_qubit]) # If we went negative, undo the subtraction by adding N
                if opt_for_vis: qc.barrier()
                qc.x(work[0])
                qc.ccx(work[0], condition_qubit, scratch) # If it's odd, then we wrapped, so clear the wrap bit
                qc.x(work[0])
                if opt_for_vis: qc.barrier()

    # Quantum Fourier Transform on precision register
    qc.append(QFT(precision_qubits), precision)

    if vis: print(qc)

    print('Circuit depth: ', qc.depth())
    print('Number of qubits: ', qc.num_qubits)

    [read_result_list, elapsed_time_quantum] = read_unsigned(N, coprime, qc, precision, classical, simulate, shots)
    #print('QPU read results: ', read_result_list)

    start_time_classical = time.time()

    repeat_period_candidates_list = estimate_num_spikes(read_result_list, 1 << precision_qubits)
    #print('Repeat period candidates list: ', repeat_period_candidates_list)

    return [repeat_period_candidates_list, elapsed_time_quantum, start_time_classical]

In [15]:
def shor_sample(N=15, coprime=2, trials=1, classical=False, simulate=True, vis=False, opt_for_vis=False):
    # N is the number we're factoring
    # The coprime needs to be 2 for the quantum implementation

    if classical: trials = 1 # the classical solution will always find the correct solution in one shot because we try all values until we find the correct ones

    print('Number to factorize: '+str(N)\
              +'\nCoprime: '+str(coprime)\
              +'\nMethod: '+str("Classical" if classical else "Simulation" if simulate else "IBM Quantum Computer")\
              +'\nNumber of trials: '+str(trials))

    [result_list, elapsed_time_quantum, start_time_classical] = Shor(N, coprime, classical, simulate, vis, opt_for_vis, trials)

    end_time_classical = time.time()
    elapsed_time_classical = end_time_classical - start_time_classical

    correct_results = len([result for result in result_list if result is not None])
    success_rate = correct_results / len(result_list)

    if success_rate > 0:
        solution = next((result for result in result_list if result is not None), None)
        solution_str = str(N)+'='+str(solution[0])+'*'+str(solution[1])
        print('Solution: '+solution_str\
              +'\nAccuracy: '+str(success_rate*100)+'%'\
              +'\nCalculation time quantum part: '+str("{:.4f}".format(elapsed_time_quantum))+'s'\
              +'\nCalculation time classical part: '+str("{:.4f}".format(elapsed_time_classical))+'s')
    else:
        print('Failure: No non-trivial factors were found.\n')

# Execution

**Here are some values of N to try:**

15, 21, 33, 35, 39, 51, 55, 69, 77, 85, 87, 91, 93, 95, 111, 115, 117,
119, 123, 133, 155, 187, 203, 221, 247, 259, 287, 341, 451

15, 21, 33, 35,     51, 55, 69,                 93, 119, 203, 341, 451, 559, 698, 815, 943,
1067, 1167, 1263, 1363, 1469, 1589, 1703

In [ ]:
# 'classical=False' executes quantum part of algorithm
# 'classical=True' executes classical alternative of quantum part for comparison
# 'simulate=True' obtains the result by running the circuit on a simulator
# 'simulate=False' obtains the result by running the circuit on a IBM quantum computer
# 'vis=True' shows quantum circuit
# 'vis=False' does not show quantum circuit
# 'opt_for_vis=True' adds barriers to the circuit to improve readability
# 'opt_for_vis=False' removes barriers in the circuit to improve performance

shor_sample(N=341, coprime=2, trials=128, classical=False, simulate=False, vis=False, opt_for_vis=False)

Number to factorize: 341
Coprime: 2
Method: IBM Quantum Computer
Number of trials: 128
Circuit depth:  35762
Number of qubits:  20
Backend:  ibm_brisbane


#Running on IBM Quantum Computer
Number to factorize: 203
Coprime: 2
Method: IBM Quantum Computer
Number of trials: 128
Circuit depth:  16405
Number of qubits:  18
Backend:  ibm_brisbane
Job ID:  cwrzvy72ac5g008hzj9g

Number to factorize: 203
Coprime: 2
Method: IBM Quantum Computer
Number of trials: 128
Circuit depth:  16405
Number of qubits:  18
Backend:  ibm_brisbane
Job ID:  cwr9800997wg008xec20

Number to factorize: 119
Coprime: 2
Method: IBM Quantum Computer
Number of trials: 128
Circuit depth:  8380
Number of qubits:  16
Backend:  ibm_brisbane
Job ID:  cwqy6ka60bqg008p6m30
Solution: 119=7*17
Accuracy: 14.84375%
Calculation time quantum part: 150s
Calculation time classical part: 0.0066s

Number to factorize: 93
Coprime: 2
Method: IBM Quantum Computer
Number of trials: 128
Circuit depth:  7170
Number of qubits:  16
Backend:  ibm_brisbane
Job ID:  cwq04v360bqg008p2pqg
Solution: 93=31*3
Accuracy: 35.9375%
Calculation time quantum part: 117s
Calculation time classical part: 0.0066s

Number to factorize: 93
Coprime: 2
Method: IBM Quantum Computer
Number of trials: 128
Circuit depth:  7170
Number of qubits:  16
Backend:  ibm_sherbrooke
Job ID:  cwpxfhytdtng0086qcqg

Number to factorize: 85
Coprime: 2
Method: IBM Quantum Computer
Number of trials: 128
Circuit depth:  5960
Number of qubits:  16
Backend:  ibm_brisbane
Job ID:  cwp9yjatdtng0086m6fg
Solution: 85=5*17
Accuracy: 28.125%
Calculation time quantum part: 115s
Calculation time classical part: 0.0146s

Number to factorize: 77
Coprime: 2
Method: IBM Quantum Computer
Number of trials: 128
Circuit depth:  6202
Number of qubits:  16
Backend:  ibm_brisbane
Job ID:  cwp9akj2ac5g008hmj00
Solution: 77=7*11
Accuracy: 19.53125%
Calculation time quantum part: 116s
Calculation time classical part: 0.0081s

Number to factorize: 69
Coprime: 2
Method: IBM Quantum Computer
Number of trials: 128
Circuit depth:  4992
Number of qubits:  16
Backend:  ibm_brisbane
Job ID:  cwp9220ehebg008hppt0
Solution: 69=23*3
Accuracy: 16.40625%
Calculation time quantum part: 107s
Calculation time classical part: 0.0080s

Number to factorize: 55
Coprime: 2
Method: IBM Quantum Computer
Number of trials: 128
Circuit depth:  3154
Number of qubits:  14
Backend:  ibm_brisbane
Job ID:  cwp81qp2ac5g008hmfdg
Solution: 55=11*5
Accuracy: 7.8125%
Calculation time quantum part: 46s
Calculation time classical part: 0.0050s

Number to factorize: 39
Coprime: 2
Method: IBM Quantum Computer
Number of trials: 128
Circuit depth:  2806
Number of qubits:  14
Backend:  ibm_brisbane
Job ID:  cwp7hcd997wg008x5e10
Solution: 39=3*13
Accuracy: 28.125%
Calculation time quantum part: 44s
Calculation time classical part: 0.0099s

Number to factorize: 35
Coprime: 2
Method: IBM Quantum Computer
Number of trials: 128
Circuit depth:  2226
Number of qubits:  14
Backend:  ibm_brisbane
Job ID:  cwp7cmj2ac5g008hmdt0
Solution: 35=7*5
Accuracy: 24.21875%
Calculation time quantum part: 38s
Calculation time classical part: 0.0051s

Number to factorize: 33
Coprime: 2
Method: IBM Quantum Computer
Number of trials: 128
Circuit depth:  1530
Number of qubits:  14
Backend:  ibm_brisbane
Job ID:  cwnwyk260bqg008nxe6g
Calculation time quantum part: 27s
Failure: No non-trivial factors were found.

Number to factorize: 21
Coprime: 2
Method: IBM Quantum Computer
Number of trials: 128
Circuit depth:  852
Number of qubits:  12
Backend:  ibm_brisbane
Job ID:  cwnwvq6997wg008x3vzg
Solution: 21=7*3
Accuracy: 40.625%
Calculation time quantum part: 10s
Calculation time classical part: 0.0027s

Number to factorize: 15
Coprime: 2
Method: IBM Quantum Computer
Number of trials: 128
Circuit depth:  418
Number of qubits:  10
Backend:  ibm_brisbane
Job ID:  cwnwstfehebg008hn2c0
Solution: 15=3*5
Accuracy: 24.21875%
Calculation time quantum part: 4s
Calculation time classical part: 0.0024s

#Running on Simulator
Number to factorize: 203
Coprime: 2
Method: Simulation
Number of trials: 128
Circuit depth:  16405
Number of qubits:  18
Solution: 203=7*29
Accuracy: 15.625%
Calculation time quantum part: 0.8556s
Calculation time classical part: 0.0169s

Number to factorize: 119
Coprime: 2
Method: Simulation
Number of trials: 128
Circuit depth:  8380
Number of qubits:  16
Solution: 119=7*17
Accuracy: 53.90625%
Calculation time quantum part: 1.1821s
Calculation time classical part: 0.0106s

Number to factorize: 93
Coprime: 2
Method: Simulation
Number of trials: 128
Circuit depth:  7170
Number of qubits:  16
Solution: 93=31*3
Accuracy: 89.0625%
Calculation time quantum part: 0.5299s
Calculation time classical part: 0.0099s

Number to factorize: 69
Coprime: 2
Method: Simulation
Number of trials: 128
Circuit depth:  4992
Number of qubits:  16
Solution: 69=23*3
Accuracy: 47.65625%
Calculation time quantum part: 0.5982s
Calculation time classical part: 0.0082s

Number to factorize: 55
Coprime: 2
Method: Simulation
Number of trials: 128
Circuit depth:  3154
Number of qubits:  14
Solution: 55=11*5
Accuracy: 32.8125%
Calculation time quantum part: 0.5277s
Calculation time classical part: 0.0163s

Number to factorize: 51
Coprime: 2
Method: Simulation
Number of trials: 128
Circuit depth:  2574
Number of qubits:  14
Solution: 51=3*17
Accuracy: 89.0625%
Calculation time quantum part: 0.2712s
Calculation time classical part: 0.0042s

Number to factorize: 39
Coprime: 2
Method: Simulation
Number of trials: 128
Circuit depth:  2806
Number of qubits:  14
Solution: 39=3*13
Accuracy: 67.96875%
Calculation time quantum part: 0.3688s
Calculation time classical part: 0.0182s

Number to factorize: 35
Coprime: 2
Method: Simulation
Number of trials: 128
Circuit depth:  2226
Number of qubits:  14
Solution: 35=7*5
Accuracy: 67.96875%
Calculation time quantum part: 0.7053s
Calculation time classical part: 0.0065s

Number to factorize: 33
Coprime: 2
Method: Simulation
Number of trials: 128
Circuit depth:  1530
Number of qubits:  14
Failure: No non-trivial factors were found.

Number to factorize: 21
Coprime: 2
Method: Simulation
Number of trials: 128
Circuit depth:  852
Number of qubits:  12
Solution: 21=7*3
Accuracy: 70.3125%
Calculation time quantum part: 0.1592s
Calculation time classical part: 0.0045s

Number to factorize: 15
Coprime: 2
Method: Simulation
Number of trials: 128
Circuit depth:  418
Number of qubits:  10
Solution: 15=3*5
Accuracy: 79.6875%
Calculation time quantum part: 0.1041s
Calculation time classical part: 0.0040s